In [1]:
%load_ext autoreload
%autoreload 2

import os
from time import time
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from causallearn.search.ConstraintBased.PC import pc
from causallearn.utils.GraphUtils import GraphUtils
from causallearn.utils.cit import CIT

import vis
import softclustering as sc
import socceraction.spadl as spadl

/Users/jgv/PycharmProjects/soccer-infogeo/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
datafolder = "data"
fifa2018h5 = os.path.join(datafolder, "spadl-fifa2018.h5")
games = pd.read_hdf(fifa2018h5, key="games")
with pd.HDFStore(fifa2018h5) as store:
    actions = []  #list of DataFrames
    for game in tqdm(games.itertuples()):
        game_action = store[f"actions/game_{game.game_id}"]
        game_action = spadl.play_left_to_right(game_action, game.home_team_id)
        game_action["is_home"] = game_action["team_id"] == game.home_team_id
        actions.append(game_action)
    actions = pd.concat(actions)
    actions.drop("original_event_id", axis=1, inplace=True)
    actions = pd.merge(actions, spadl.config.actiontypes_df(), how="left")
    actions = sc.consolidate(actions)
    actions = sc.add_noise(actions)
    actions = sc.remove_outliers(actions, True)
    actions["angle"] = np.arctan2(actions.end_y - actions.start_y, actions.end_x - actions.start_x)
    actions["cos_angle"] = np.cos(actions["angle"])
    actions["sin_angle"] = np.sin(actions["angle"])

64it [00:00, 166.73it/s]


Remove 555 out of 128484 datapoints.


In [3]:
no_dir_action_types = set(actions[actions["angle"] == 0.0]["type_name"])
action_types = set(actions["type_name"]) - no_dir_action_types - {"shot_penalty"}

for alpha in [0.01, 0.05, 0.10, 0.15]:
    for action in action_types:
        print(f"Processing {action} with alpha={alpha}")
        mask = (actions["type_name"]==action) & (actions["team_id"]==771)
        columns = ["start_x", "start_y","cos_angle", "sin_angle"]
        #columns = ["start_x", "start_y","angle"]
        data = actions[mask][columns]
        cg = pc(data.to_numpy(),
                    alpha=alpha,
                    indep_test="fastkci",
                    kernelX='Gaussian',
                    kernelY='Gaussian',
                    est_width='empirical',
                    show_progress=False,
                )
        pyd = GraphUtils.to_pydot(cg.G, labels=columns)
        pyd.write_pdf(f'plots/DAG_{action}_{str(alpha)}.pdf')

Processing clearance with alpha=0.01

Processing shot with alpha=0.01

Processing dribble with alpha=0.01



/Users/jgv/PycharmProjects/soccer-infogeo/.venv/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Processing throw_in with alpha=0.01



/Users/jgv/PycharmProjects/soccer-infogeo/.venv/lib/python3.11/site-packages/causallearn/utils/FastKCI/FastKCI.py:164: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  data_z = stats.zscore(data_z, ddof=1, axis=0)
/Users/jgv/PycharmProjects/soccer-infogeo/.venv/lib/python3.11/site-packages/causallearn/utils/FastKCI/FastKCI.py:164: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  data_z = stats.zscore(data_z, ddof=1, axis=0)
/Users/jgv/PycharmProjects/soccer-infogeo/.venv/lib/python3.11/site-packages/causallearn/utils/FastKCI/FastKCI.py:164: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  data_z = stats.zscore(data_z, dd

Processing freekick with alpha=0.01

Processing cross with alpha=0.01

Processing corner with alpha=0.01



...

Processing freekick with alpha=0.15



/Users/jgv/PycharmProjects/soccer-infogeo/.venv/lib/python3.11/site-packages/causallearn/utils/FastKCI/FastKCI.py:470: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  data = stats.zscore(data, ddof=1, axis=0)
/Users/jgv/PycharmProjects/soccer-infogeo/.venv/lib/python3.11/site-packages/causallearn/utils/FastKCI/FastKCI.py:470: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  data = stats.zscore(data, ddof=1, axis=0)
/Users/jgv/PycharmProjects/soccer-infogeo/.venv/lib/python3.11/site-packages/causallearn/utils/FastKCI/FastKCI.py:470: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  data = stats.zscore(data, ddof=1, axis=0

Processing cross with alpha=0.15

Processing corner with alpha=0.15



/Users/jgv/PycharmProjects/soccer-infogeo/.venv/lib/python3.11/site-packages/causallearn/utils/FastKCI/FastKCI.py:470: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  data = stats.zscore(data, ddof=1, axis=0)
/Users/jgv/PycharmProjects/soccer-infogeo/.venv/lib/python3.11/site-packages/causallearn/utils/FastKCI/FastKCI.py:470: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  data = stats.zscore(data, ddof=1, axis=0)
/Users/jgv/PycharmProjects/soccer-infogeo/.venv/lib/python3.11/site-packages/causallearn/utils/FastKCI/FastKCI.py:470: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  data = stats.zscore(data, ddof=1, axis=0

Processing goalkick with alpha=0.15

Processing keeper_action with alpha=0.15

Processing pass with alpha=0.15

